In [1]:
from keras.applications import ResNet50
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from imutils import paths
import numpy as np
import argparse
import random
import os

Using TensorFlow backend.


In [ ]:
from hdf5datasetwriter import HDF5DatasetWriter

In [2]:
import h5py

In [ ]:
imagePaths = list(paths.list_images("/home/divyanshu/Desktop/Kaggle/train"))
random.shuffle(imagePaths)
labels = [p.split(os.path.sep)[-1].split(".")[0] for p in imagePaths]
le = LabelEncoder()
labels = le.fit_transform(labels)
print(labels[0])

In [4]:
model = ResNet50(weights = "imagenet", include_top = False)

In [6]:
dataset = HDF5DatasetWriter((len(imagePaths), 2048), "/home/divyanshu/Desktop/Kaggle/outputs", dataKey = "features")
dataset.storeClassLabels(le.classes_)

In [7]:
print(len(imagePaths))

25000


In [9]:
for i in np.arange(0, len(imagePaths), 32):
    batchPaths = imagePaths[i: i + 32]
    batchLabels = labels[i: i + 32]
    batchImages = []
    for (j, imagePath) in enumerate(batchPaths):
        image = load_img(imagePath, target_size = (224, 224))
        image = img_to_array(image)
        image = np.expand_dims(image, axis = 0)
        image = imagenet_utils.preprocess_input(image)
        batchImages.append(image)
    batchImages = np.vstack(batchImages)
    features = model.predict(batchImages, batch_size = 32)
    features = features.reshape((features.shape[0], 2048))
    dataset.add(features, batchLabels)
    print(i)
dataset.close()

0
32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
2464
2496
2528
2560
2592
2624
2656
2688
2720
2752
2784
2816
2848
2880
2912
2944
2976
3008
3040
3072
3104
3136
3168
3200
3232
3264
3296
3328
3360
3392
3424
3456
3488
3520
3552
3584
3616
3648
3680
3712
3744
3776
3808
3840
3872
3904
3936
3968
4000
4032
4064
4096
4128
4160
4192
4224
4256
4288
4320
4352
4384
4416
4448
4480
4512
4544
4576
4608
4640
4672
4704
4736
4768
4800
4832
4864
4896
4928
4960
4992
5024
5056
5088
5120
5152
5184
5216
5248
5280
5312
5344
5376
5408
5440
5472
5504
5536
5568
5600
5632
5664
5696
5728
5760
5792
5824
5856
5888
5920
5952
5984
6016
6048
6080
6112
6144
6176
6208
6240
6272
6304
6336
6368
6400
6432
6464
6496
6528
6560
6592
66

In [3]:
#Logistic Regression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

In [4]:
db = h5py.File("/home/divyanshu/Desktop/Kaggle/Cats vs Dogs/outputs", "r")
i = int(db["labels"].shape[0]*0.75)

In [7]:
params = {"C": [0.0001, 0.001, 0.01, 0.1, 1.0]}
reg = LogisticRegression(C = 0.01)
reg.fit(db["features"][:i], db["labels"][:i])

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
params = {"C": [0.01, 0.1, 1.0]}
svm = GridSearchCV(LinearSVC(), params, cv = 3)
svm.fit(db["features"][:i], db["labels"][:i])

GridSearchCV(cv=3, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.01, 0.1, 1.0]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [12]:
print(svm.best_params_)

{'C': 0.01}


In [13]:
pred = svm.predict(db["features"][i:])
print(classification_report(db["labels"][i:], pred, target_names=db["label_names"]))
acc = accuracy_score(db["labels"][i:], pred)
print(acc)

             precision    recall  f1-score   support

        cat       0.99      0.99      0.99      3094
        dog       0.99      0.99      0.99      3156

avg / total       0.99      0.99      0.99      6250

0.98912


In [14]:
p = reg.predict_proba(db["features"][i:])
print(p[0])

[5.12217206e-04 9.99487783e-01]


In [8]:
pred = reg.predict(db["features"][i:])
print(classification_report(db["labels"][i:], pred, target_names=db["label_names"]))
acc = accuracy_score(db["labels"][i:], pred)
print(acc)

             precision    recall  f1-score   support

        cat       0.99      0.99      0.99      3094
        dog       0.99      0.99      0.99      3156

avg / total       0.99      0.99      0.99      6250

0.98944


In [15]:
from sklearn.metrics import log_loss
log_loss(db["labels"][i:], p)

0.029493557666077648

In [20]:
import pickle

In [21]:
f = open("/home/divyanshu/Desktop/Kaggle/trained_model/dogsvscats.pickle", "wb")
f.write(pickle.dumps(reg.best_estimator_))
f.close()
db.close()